## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Calibrating the camera using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [2]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)

In [3]:
vidcap = cv2.VideoCapture('../project_video.mp4')
images =[]
ret = True
while(ret):
    ret, frame = vidcap.read()
    
    if(ret == False):
        break
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    images.append(img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
vidcap.release()

In [4]:
def undistort_images(images, mtx, dist):
    undists = []
    for i in range(0, len(images)):
        img = images[i]
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        undists.append(undist)
        
    return undists

In [5]:
undists = undistort_images(images, mtx, dist)

In [6]:
plt.figure(figsize=(15,4))
plt.subplot(121)
plt.imshow(images[0])
plt.title("distorted image")
plt.tight_layout()
plt.subplot(122)
plt.imshow(undists[0])
plt.title("undistorted image")
plt.tight_layout()

## Binary image

In [7]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary
    

In [8]:
bins =[]
numImage = len(undists)

for i in range(0, numImage):
    binary = pipeline(undists[i])
    bins.append(binary)


In [9]:
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
f.tight_layout()

ax1.imshow(undists[0])
ax1.set_title('undistorted image')

ax2.imshow(bins[0])
ax2.set_title('binary result')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.1)

## Perspective transform

In [50]:
straightFrame = 350
plt.imshow(undists[straightFrame])

In [51]:
dstOffset = 150
img_size = (undists[0].shape[1], undists[0].shape[0])

src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])


M = cv2.getPerspectiveTransform(src, dst)        

In [55]:
# warped image 
warped_images = []
for i in range(0,numImage):
    warped = cv2.warpPerspective(bins[straightFrame], M, img_size)
    warped_images.append(warped)


In [82]:
# Plot the result
import matplotlib.patches as patches
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
f.tight_layout()

ax1.imshow(bins[straightFrame], cmap ='gray')
ax1.set_title('Thresholded image with source points drawn')
srcPoly = patches.Polygon(src,linewidth=2,edgecolor='r', facecolor = 'none')
ax1.add_patch(srcPoly)

ax2.imshow(warped_images[straightFrame], cmap ='gray')
ax2.set_title('Warped result with dest.points drawn')
dstPoly = patches.Polygon(dst,linewidth=2,edgecolor='r', facecolor = 'none')
ax2.add_patch(dstPoly)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.1)

## Finding the lines

In [60]:
# plot histogram
img = warped_images[straightFrame]
histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
plt.plot(histogram)

In [61]:
binary_warped = warped_images[straightFrame]
histogram = np.sum(img[img.shape[0]//2:,:], axis=0)

out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint


In [66]:
# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(binary_warped.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

In [70]:
# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = binary_warped.shape[0] - (window+1)*window_height
    win_y_high = binary_warped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
    (0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
    (0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))


In [72]:
# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

In [75]:
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

In [80]:
binary_warped =  warped_images[straightFrame+1]

nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
margin = 100
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
left_fit[1]*nonzeroy + left_fit[2] + margin))) 

right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [81]:
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')


In [ ]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #count 
        self.count = 0
        
        # meters per pixel in x dimension
        self.xm_per_pix = 3.7/980
        
    def LineDetection(self, binary_warped, nwindows = 9, margin = 100, minpix = 50):
        
        height = binary_warped.shape[0]
        width = binary_warped.shape[1]
        
        self.allx, self.ally = self.LinePointsDetection(binary_warped, nwindows, margin, minpix) 
        self.current_fit = self.SecondPolyFitting(self.allx, self.ally)
 
        if(self.current_fit[0] == False):
            self.best_fit = self.current_fit
            self.detected = True
        else:  
            self.detected = self.SanityCheck(width, height, currentFit_candi, self.best_fit)
        
        if(self.detected):
            self.RecentPointsStacking(self.recent_xfitted, self.allx)
            
        self.radius_of_curvature = self.RadiusMeasuringInMeters( self.allx, binary_warped.shape[0])
        self.line_base_pos = self.CenterPositionInMeters(width, height, self.best_fit)
        
        
    def LinePointsDetection(self, binary_warped, nwindows = 9, margin = 100, minpix = 50):
        
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        if(!self.detected):
            # Set height of windows
            window_height = np.int(binary_warped.shape[0]/nwindows)
            # Current positions to be updated for each window
            leftx_current = leftx_base
            rightx_current = rightx_base
            # Create empty lists to receive left and right lane pixel indices
            left_lane_inds = []
            right_lane_inds = []
            # Step through the windows one by one
            for window in range(nwindows):
                # Identify window boundaries in x and y (and right and left)
                win_y_low = binary_warped.shape[0] - (window+1)*window_height
                win_y_high = binary_warped.shape[0] - window*window_height
                win_xleft_low = leftx_current - margin
                win_xleft_high = leftx_current + margin
                win_xright_low = rightx_current - margin
                win_xright_high = rightx_current + margin
                # Draw the windows on the visualization image
                cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
                (0,255,0), 2) 
                cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
                (0,255,0), 2) 
                # Identify the nonzero pixels in x and y within the window
                good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
                good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
                # Append these indices to the lists
                left_lane_inds.append(good_left_inds)
                right_lane_inds.append(good_right_inds)
                # If you found > minpix pixels, recenter next window on their mean position
                if len(good_left_inds) > minpix:
                    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
                if len(good_right_inds) > minpix:        
                    rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            # Concatenate the arrays of indices
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)

        else:
            left_fit = self.current_fit[0]
            right_fit = self.current_fit[1]
            
            left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
            left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
            left_fit[1]*nonzeroy + left_fit[2] + margin))) 

            right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
            right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
            right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        #left_fit = np.polyfit(lefty, leftx, 2)
        #right_fit = np.polyfit(righty, rightx, 2)

        return [leftx, rightx], [lefty, righty]
    
    def SecondPolyFitting(xPoints, yPoints):
        leftx = xPoints[0]
        rightx = xPoints[1]
        lefty = yPoints[0]
        righty = yPoints[1]
        
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        return [left_fit, right_fit]
        
    def RecentPointsStacking(self,recentNXPoints, xPoints, yPoints, numFits = 10):
        leftx = xPoints[0]
        rightx = xPoints[1]
        
        lefty = yPoints[0]
        righty = yPoints[1]
        
        if(self.count > numFits):
            cumLeftx = recentNXPoints[0]
            cumRightx = recentNXPoints[1]
            
            cumLefty = recentNYPoints[0]
            cumRighty = recentNYPoints[1]
        
            meanLeftLen = len(cumLeftx)/float(numFits)
            meanRightLen = len(cumRightx)/float(numFits)
            recentNXPoints[0] = cumLeftx[1:len(cumLeftx)]
            recentNXPoints[1] = cumRightx[1:len(cumRightx)]
            
            meanLeftLen = len(cumLefty)/float(numFits)
            meanRightLen = len(cumRighty)/float(numFits)
            recentNYPoints[0] = cumLefty[1:len(cumLefty)]
            recentNYPoints[1] = cumRighty[1:len(cumRighty)]
            
        # Append the new points to the lists
        recentNXPoints[0].append(leftx)
        recentNXPoints[1].append(rightx)
        
        recentNYPoints[0].append(leftx)
        recentNYPoints[1].append(rightx)
        
        self.count += 1
        
        return 
            
        
        
    def RadiusMeasuringInMeters(self, xPoints, yRange):
        leftx = xPoints[0]
        rightx = xPoints[1]
        ploty = np.linspace(0, yRange-1, num= yRange )# to cover same y-range as image
        
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 3.0/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
        
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
        return [left_curverad, right_curverad]
    
    def CenterPositionInMeters(self, xRange, yRange, fit):
        carPosition = xRange/2
        left_fit = fit[0]
        right_fit = fit[1]
        
        leftBottomPos = left_fit[0]*yRange**2 + left_fit[1]*yRange + left_fit[2]
        rightBottomPos = right_fit[0]*yRange**2 + right_fit[1]*yRange + right_fit[2]
        
        laneCenterPos = (leftBottomPos + rightBottomPos) /2
        centerDist = (carPosition - laneCenterPos) * self.xm_per_pix
        
        return centerDist
        
    def SanityCheck(self, xRange, yRange, currFit, prevFit):
        
        sanityFlag = False
        
        centerTol = 0.3
        coeffTol = [0.1, 0.1, 100]
        
        #check center position
        centerDist = self.CenterPositionInMeters(xRange, yRange, currFit)
        #check difference with prev fit
        leftDiffCoeff = np.abs(prevFit[0]-currFit[0])
        rightDiffCoeff = np.abs(prevFit[1]-currFit[1])
        
        if( centerDist < centerTol 
          or leftDiffCoeff < coeffTol
          or rightDiffCoeff < coeffTol):
            sanityFlag = True
        
        return sanityFlag

        

array([ -9.66344485e-05,   7.24702777e-02,   1.41230650e+02])